## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


In [4]:

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/cities2.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bandar,ID,2.0500,99.7500,77.99,87,100,1.70,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,85.28,80,100,8.05,light rain
2,2,Leningradskiy,RU,69.3833,178.4167,6.28,91,90,14.38,overcast clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,-2.07,93,100,5.99,overcast clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,50.11,87,20,26.46,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
print(min_temp)
print(max_temp)

75.0
90.0


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
df = city_data_df.loc[city_data_df["Max Temp"]<float(max_temp)]
df = city_data_df.loc[city_data_df["Max Temp"]>float(min_temp)]

df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bandar,ID,2.0500,99.7500,77.99,87,100,1.70,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,85.28,80,100,8.05,light rain
7,7,Kapaa,US,22.0752,-159.3190,82.38,74,75,14.97,broken clouds
10,10,Faanui,PF,-16.4833,-151.7500,81.41,73,17,14.03,few clouds
12,12,Laurel,US,39.0993,-76.8483,80.06,49,0,5.75,clear sky
13,13,Atuona,PF,-9.8000,-139.0333,78.98,78,69,18.30,light rain
23,23,Bodden Town,KY,19.2833,-81.2500,81.00,77,20,5.75,few clouds
25,25,Touros,BR,-5.1989,-35.4608,78.85,83,7,8.21,clear sky
26,26,Agutaya,PH,11.1480,120.9417,80.91,78,100,8.34,overcast clouds
28,28,Rikitea,PF,-23.1203,-134.9692,77.85,71,12,18.34,light rain


In [8]:
# 4a. Determine if there are any empty rows.
df.isnull().values.sum()



0

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.



In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bandar,ID,77.99,overcast clouds,2.0500,99.7500,
1,Hilo,US,85.28,light rain,19.7297,-155.0900,
7,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,
10,Faanui,PF,81.41,few clouds,-16.4833,-151.7500,
12,Laurel,US,80.06,clear sky,39.0993,-76.8483,
13,Atuona,PF,78.98,light rain,-9.8000,-139.0333,
23,Bodden Town,KY,81.00,few clouds,19.2833,-81.2500,
25,Touros,BR,78.85,clear sky,-5.1989,-35.4608,
26,Agutaya,PH,80.91,overcast clouds,11.1480,120.9417,
28,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        
        

Hotel not found... skipping.0
Hotel not found... skipping.26
Hotel not found... skipping.62
Hotel not found... skipping.130
Hotel not found... skipping.236
Hotel not found... skipping.241
Hotel not found... skipping.488
Hotel not found... skipping.525
Hotel not found... skipping.526
Hotel not found... skipping.538
Hotel not found... skipping.639
Hotel not found... skipping.645
Hotel not found... skipping.672


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bandar,ID,77.99,overcast clouds,2.0500,99.7500,
1,Hilo,US,85.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Faanui,PF,81.41,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Laurel,US,80.06,clear sky,39.0993,-76.8483,Days Inn & Suites by Wyndham Laurel Near Fort ...
...,...,...,...,...,...,...,...
666,Ixtapa,MX,76.23,scattered clouds,20.7000,-105.2000,hotel la terminal
667,Kalmunai,LK,80.65,broken clouds,7.4167,81.8167,VS Villa
669,Domoni,KM,78.82,broken clouds,-12.2569,44.5319,Al Djadid
672,Chihuahua,MX,76.03,clear sky,28.5000,-106.0000,


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))